In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [7]:
print(torch.__version__)

2.7.0+cu126


In [8]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader

# Load Iris dataset
iris = load_iris()
X = iris['data']
y = iris['target']

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Convert to torch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# Create TensorDatasets
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)


In [ ]:
# Split the training set further into train and validation sets
X_train_sub, X_val, y_train_sub, y_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42, stratify=y_train
)

# Convert to torch tensors
X_train_sub_tensor = torch.tensor(X_train_sub, dtype=torch.float32)
y_train_sub_tensor = torch.tensor(y_train_sub, dtype=torch.long)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.long)

# Create TensorDataset and DataLoader for validation set
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

# Update train_dataset and train_loader to use the new training split
train_dataset = TensorDataset(X_train_sub_tensor, y_train_sub_tensor)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

In [9]:
class IrisNet(nn.Module):
    def __init__(self):
        super(IrisNet, self).__init__()
        self.fc1 = nn.Linear(4, 16)
        self.fc2 = nn.Linear(16, 3)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = IrisNet()
model

IrisNet(
  (fc1): Linear(in_features=4, out_features=16, bias=True)
  (fc2): Linear(in_features=16, out_features=3, bias=True)
)

In [10]:
# Example training and accuracy evaluation code for the IrisNet model

# Assume you have train_loader and test_loader DataLoader objects ready
# Each batch should return (inputs, labels), where:
# - inputs: torch.Tensor of shape [batch_size, 4]
# - labels: torch.Tensor of shape [batch_size] with class indices (0, 1, or 2)
# Validation function


import torch.optim as optim

# Hyperparameters
epochs = 10
learning_rate = 0.01

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader):.4f}")

# Accuracy evaluation
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
accuracy = 100 * correct / total
print(f"Test Accuracy: {accuracy:.2f}%")

Epoch 1/10, Loss: 1.1387
Epoch 2/10, Loss: 0.9282
Epoch 3/10, Loss: 0.7728
Epoch 4/10, Loss: 0.6285
Epoch 5/10, Loss: 0.4945
Epoch 6/10, Loss: 0.4468
Epoch 7/10, Loss: 0.3845
Epoch 8/10, Loss: 0.3321
Epoch 9/10, Loss: 0.3147
Epoch 10/10, Loss: 0.3232
Test Accuracy: 93.33%


In [11]:
def evaluate(model, val_loader):
    model.eval()
    correct = 0
    total = 0
    val_loss = 0.0
    with torch.no_grad():
        for inputs, labels in val_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    avg_loss = val_loss / len(val_loader)
    accuracy = 100 * correct / total
    return avg_loss, accuracy
